In [1]:
import numpy as np
import pandas as pd

In [137]:
def analyse_diffraction(data, lamda, angle=2):
    
    decimals = len(str(lamda).split('.')[1])
    
    #first change data into a usable format
    if type(data) is str:
        
        if data.find('\n') != -1:
            data = data.splitlines()
            data = np.array([float(i.replace(',', '').strip()) for i in data])
            
        else:
            data = data.replace(',', '').split(' ')
            data = np.array(list(filter(lambda x: x != ' ', data) ))
    
    else:
        data = np.array(data)
    
    d = np.round(lamda / (2 * np.sin(np.radians(data / angle) ) ), decimals)
    cubicd = np.round(1 / d ** 2, decimals)
    const = np.round(cubicd / np.amin(cubicd), 3)
    
    name1 = r'$\frac{1}{d^{2}}$'
    name2 = r'$k^{2}$ + $l^{2}$ + $m^{2}$'
    
    if (const - np.round(const)  < 0.02).all():
        const = np.round(const).astype(np.uint16)
        return pd.DataFrame({'2θ': data, 'd /Å': d, name1: cubicd, name2: const})
        #add miller planes
    else:
        
        for i in range(2, 101):
            
            const2 = const * i
            if (const2 - np.round(const2)  < 0.05).all():
                break
        
        const2 = np.round(const2).astype(np.uint16)
        return pd.DataFrame({'2θ': data, 'd /Å': d, name1: cubicd, 'const': const, name2: const2})

In [138]:
analyse_diffraction([27.38, 31.72, 45.47, 53.89, 56.50, 66.26, 75.33], 1.5418)

,2θ,d /Å,$\frac{1}{d^{2}}$,const,$k^{2}$ + $l^{2}$ + $m^{2}$
0,27.38,3.2573,0.0943,1.000,3
1,31.72,2.8208,0.1257,1.333,4
2,45.47,1.9947,0.2513,2.665,8
3,53.89,1.7013,0.3455,3.664,11
4,56.50,1.6287,0.3770,3.998,12
5,66.26,1.4105,0.5026,5.330,16
6,75.33,1.2616,0.6283,6.663,20


In [139]:
analyse_diffraction([27.46,   39.22,   48.54,   56.68,   64.10,   71.09,   84.33], 1.5418)

,2θ,d /Å,$\frac{1}{d^{2}}$,name2
0,27.46,3.2480,0.0948,1
1,39.22,2.2970,0.1895,2
2,48.54,1.8755,0.2843,3
3,56.68,1.6240,0.3792,4
4,64.10,1.4527,0.4739,5
5,71.09,1.3261,0.5687,6
6,84.33,1.1484,0.7583,8
